In [18]:
def get_points_code(N,r,F):
    points = []
    traces = {} # элементы со следом j
    traces[0] = [a for a in F if a.trace() == 0]
    for B in F:
        if len(points) >= N:
            return points
        if B not in traces[0]:
            c = 2*(B^(r+1)).trace() / B.trace()
            if c not in traces:
                traces[c] = [a for a in F if a.trace() == c]
            for j in traces[c]:
                if len(points) >= N:
                    return points
                points.append([B, j])
    return points

# Автоморфизм кривой
def aut_sigma(xi,i,r):
    return xi * alpha^((r+1)*r^(i-1))

def FGS_encode(N,l,r,e,m,s,F,T2,f):
    n = m*N
    print('Длина кода n =', n)
    q = r^e
    points_code = get_points_code(N,r,F)
    print("Точки кода:", points_code)
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
    print("L(l*𝑃∞):", L1)
    k = len(L1)
    print('Размерность кода k =', k)
    print('Скорость кода R >=', round(float(k/n),5))
    d = N-l/m
    print('Минимальное расстояние кода d >=', d)
    t_min = int((d-1)/2)
    t_max = int(float(N*(m-1) + N*s/(s+1)*(1 - k/(N*(m-s+1))) - 3*(l+1-k)/(m-s+1)))
    print('Число исправляемых ошибок:', t_min, '<= t <=', t_max)
    print('Исходное сообщение f =', f)
    f_RR = 0
    for f_el, L1_el in zip(f, L1):
        f_RR += f_el*L1_el
    PR = PolynomialRing(F, 2, ['x1','x2'])
    f_RR = PR(f_RR)
    print('Исходное сообщение как элемент L(l*𝑃∞):', f_RR)
    y = [[0 for i in range(0,N)] for j in range(0,m)]
    for i in range(0,m):
        for j in range(0,N):
            x_1 = points_code[j][0]
            x_2 = points_code[j][1]
            for count_sigma in range(0,i):
                x_1 = aut_sigma(x_1,1,r)
                x_2 = aut_sigma(x_2,2,r)
            y[i][j] = f_RR(x_1, x_2)
    return y

def FGS_decode(N,l,e,r,m,s,F,T2,y):
    P_inf = T2.places_infinite()[0]
    L1 = (l*P_inf).basis_function_space() # Базис пространства L(l*𝑃∞)
    print('L(l*𝑃∞):', L1)
    k = len(L1)
    D = int((N*(m-s+1)-k+s*(l+1-k)+1)/(s+1)) # Степенной параметр
    L2 = (D*P_inf).basis_function_space() # Базис пространства L(D*𝑃∞)
    print('L(D*𝑃∞):', L2)
    L3 = ((l+D)*P_inf).basis_function_space() # Базис пространства L((l+D)*𝑃∞)
    print('L((l+D)*𝑃∞):', L3)
    points_code = get_points_code(N,r,F)
    count_lines = N*(m-s+1)
    count_columns = len(L3)+len(L2)
    A0_A1_matrix = matrix(F, count_lines, count_columns) # Матрица коэффициентов
    PR = PolynomialRing(F, 2, ['x1','x2'])
    for j in range(0,N):
        x_1 = points_code[j][0]
        x_2 = points_code[j][1]
        for count_sigma in range(0,m-s+1):
            equation_coefficients = []
            for i in range(0,len(L3)):
                L3_i = PR(L3[i])
                equation_coefficients.append(L3_i(x_1, x_2))
            for i in range(0,len(L2)):
                L2_i = PR(L2[i])
                equation_coefficients.append(L2_i(x_1, x_2)*y[count_sigma][j])
            A0_A1_matrix[j*(m-s+1)+count_sigma] = equation_coefficients
            x_1 = aut_sigma(x_1,1,r)
            x_2 = aut_sigma(x_2,2,r)
    x = A0_A1_matrix.right_kernel().basis()[0]
    print("Матрица для нахождения A0 и A1:")
    print(A0_A1_matrix)
    print("Частное решение:",x) 
    _.<Y> = PolynomialRing(F)
    A0_coeff = x[:len(L3)]
    A1_coeff = x[len(L3):]
    A0 = 0
    A1 = 0
    for coeff, L3_elem in zip(A0_coeff, L3):
        A0 += coeff*L3_elem
    for coeff, L2_elem in zip(A1_coeff, L2):
        A1 += coeff*L2_elem
    Q = A0 + A1*Y
    print("A0 =", A0)
    print("A1 =", A1)
    print("Q =", Q)
    
    count_lines = N
    count_columns = len(L1)
    f_matrix = matrix(F, count_lines, count_columns)
    b = [0]*count_lines
    A0_PR = PR(A0)
    A1_PR = PR(A1)
    for j in range(0,count_lines):
        x_1 = points_code[j][0]
        x_2 = points_code[j][1]
        equation_coefficients = []
        for i in range(0,count_columns):
            L1_i = PR(L1[i])
            equation_coefficients.append(L1_i(x_1, x_2)*A1_PR(x_1,x_2))
        f_matrix[j] = equation_coefficients
        b[j] = -A0_PR(x_1,x_2)
    b_vector = vector(b)
    print("Матрица для нахождения исходного сообщения:")
    for i in f_matrix:
        print(i)
    print("Столбец свободных членов:", b_vector)
    try:
        recovered_message = f_matrix.solve_right(b_vector)
        return list(recovered_message)
    except:
        return "Декодирование невозможно"

    
    

N = 9 # Кол-во точек кода
l = 8 # Множитель дивизора (l*𝑃∞)
r = 3 # Характеристика поля констант F
e = 2 # Ступень башни Г-Ш
m = 2 # 1 <= m <= r-1 (число вычисляемых автоморфизмов для каждой точки при кодировании)
s = 1 # s >= 1 (константа декодера)
F.<alpha> = GF(r^e) # Базовое поле
T1.<x1> = FunctionField(F); _.<Y> = T1[] # Функциональное поле (первая ступень башни Г-Ш)
T2.<x2> = T1.extension(Y^r + Y - (x1^r / (x1^(r-1) + 1))) # Расширение функционального поля (вторая ступень башни Г-Ш)


f = [1, 2, alpha, alpha+1, alpha+2] # Сообщение

print("Кодирование:")
f_encode = FGS_encode(N,l,r,e,m,s,F,T2,f)
print("Кодовое слово:", f_encode)
# Внесём ошибки
f_encode[0][0] = 0
f_encode[1][1] = alpha+2
print("Кодовое слово с ошибками:", f_encode)
print("------------------------------------------------------")
print("Декодирование:")
recovered_message = FGS_decode(N,l,e,r,m,s,F,T2,f_encode)

if (f == recovered_message):
    print("Восстановленное сообщение:", recovered_message)
    print("Сообщение успешно декодировано!")
else:
    print(f'Ошибка: {recovered_message}')


Кодирование:
Длина кода n = 18
Точки кода: [[alpha, 2*alpha], [alpha, alpha + 2], [alpha, 1], [2*alpha + 1, 2*alpha], [2*alpha + 1, alpha + 2], [2*alpha + 1, 1], [2, alpha], [2, 2*alpha + 1], [2, 2]]
L(l*𝑃∞): [x1^2, x1, 1, (x1^2 + 1)*x2, (x1^2 + 1)*x2^2]
Размерность кода k = 5
Скорость кода R >= 0.27778
Минимальное расстояние кода d >= 5
Число исправляемых ошибок: 2 <= t <= 6
Исходное сообщение f = [1, 2, alpha, alpha + 1, alpha + 2]
Исходное сообщение как элемент L(l*𝑃∞): (alpha - 1)*x1^2*x2^2 + (alpha + 1)*x1^2*x2 + x1^2 + (alpha - 1)*x2^2 - x1 + (alpha + 1)*x2 + (alpha)
Кодовое слово: [[1, alpha + 1, alpha, 2, alpha + 1, 2*alpha + 2, alpha + 1, 2, alpha + 1], [alpha, 2, alpha, alpha, alpha + 2, alpha, 2*alpha + 1, alpha, 2*alpha]]
Кодовое слово с ошибками: [[0, alpha + 1, alpha, 2, alpha + 1, 2*alpha + 2, alpha + 1, 2, alpha + 1], [alpha, alpha + 2, alpha, alpha, alpha + 2, alpha, 2*alpha + 1, alpha, 2*alpha]]
------------------------------------------------------
Декодирование:
L(l